In [10]:
# 중심축 기반 부피 계산 함수수
import trimesh
import numpy as np
from scipy.spatial.transform import Rotation
from scipy.optimize import least_squares

# 상수 정의
THICKNESS = 0.0003 ##단면의 두께
MIN_POINTS_PER_SLICE = 10 ##원 검출 시 최소 포인트 수
MAX_CONSECUTIVE_FAILURES = 3 ##원 검출 시 최대 연속 실패 횟수
MIN_RADIUS, MAX_RADIUS = 0.02, 0.1 ##원 검출 시 최소 및 최대 반지름
MIN_POINTS_PER_METER = 100.0 ##중심축 기반 원검출시 최소 포인트 수


# 바닥 검출 파라미터
INNER_RADIUS_RATIO = 0.6      # 원 반지름의 60% 이내를 내부로 간주
MIN_INNER_DENSITY = 400000.0      # 최소 밀도 (points/m²)
MIN_INNER_RATIO = 0.08        # 최소 비율 (전체의 8%)
BOTTOM_SEARCH_RATIO = 0.2     # 하위 20%만 바닥 검출

# GLB 로드 및 z축이 중력방향이 되도록 scene을 회전하는 함수
def load_and_align_scene(filepath):
    scene = trimesh.load(filepath)
    A = np.array(scene.metadata.get('hf_alignment', np.eye(4))).reshape(4, 4)
    gravity = (A[:3, :3] @ np.array([[1,0,0],[0,-1,0],[0,0,-1]]) @ np.array([0,1,0]))
    gravity = gravity / np.linalg.norm(gravity)
    
    R = Rotation.align_vectors([[0, 0, 1]], [gravity])[0]
    T = np.eye(4)
    T[:3, :3] = R.as_matrix()
    for geom in scene.geometry.values():
        geom.apply_transform(T)
    
    for geometry in scene.geometry.values():
        if isinstance(geometry, trimesh.PointCloud):
            return geometry.vertices
    return None


# 원 검출 함수들


# 인라이너 포인트들을 받아서 원의 중심과 반지름을 정확하게 추정하는 함수
def refine_circle_ls(inlier_points):
    if len(inlier_points) < 3:
        return None
    center_init = inlier_points.mean(axis=0)
    radius_init = np.mean(np.linalg.norm(inlier_points - center_init, axis=1))
    
    def residuals(params):
        cx, cy, r = params
        return np.linalg.norm(inlier_points - [cx, cy], axis=1) - r
    
    try:
        result = least_squares(residuals, [center_init[0], center_init[1], radius_init],
                              bounds=([-0.5, -0.5, 0.001], [0.5, 0.5, 0.2]))
        cx, cy, r = result.x
        if 0.01 < r < 0.15:
            return {'center': np.array([cx, cy]), 'radius': r}
    except:
        pass
    return None

# 랜덤 샘플링을 통해 원을 추정하는 함수.한 단면에서 최적의 원을 찾을 때는 단순히 가장 원다운 원(인라이너 비율)을 찾음.
# 여러 단면에서 나온 원들 중에에 중심축을 추정하기 위한 단면을 고를때는 원점과 원중심까지의 거리를 고려해서 상위 5개의 단면의 원들의 중앙값 사용용
def fit_circle_ransac(points_2d, n_iter=50, threshold=0.002, min_inliers=10):
    if len(points_2d) < 3:
        return None
    
    best_circle, best_score = None, 0
    for _ in range(n_iter):
        p1, p2, p3 = points_2d[np.random.choice(len(points_2d), 3, replace=False)]
        try:
            A = np.array([[2*(p2[0]-p1[0]), 2*(p2[1]-p1[1])],
                         [2*(p3[0]-p1[0]), 2*(p3[1]-p1[1])]])
            b = np.array([p2[0]**2 - p1[0]**2 + p2[1]**2 - p1[1]**2,
                         p3[0]**2 - p1[0]**2 + p3[1]**2 - p1[1]**2])
            center = np.linalg.solve(A, b)
            radius = np.linalg.norm(p1 - center)
            
            if not (0.01 < radius < 0.15):
                continue
            
            distances = np.abs(np.linalg.norm(points_2d - center, axis=1) - radius)
            inliers = distances < threshold
            n_inliers = np.sum(inliers)
            
            if n_inliers < min_inliers:
                continue
            
            score = n_inliers / len(points_2d)
            if score > best_score:
                best_score = score
                refined = refine_circle_ls(points_2d[inliers])
                if refined:
                    best_circle = {**refined, 'score': score, 'n_inliers': n_inliers}
        except np.linalg.LinAlgError:
            continue
    return best_circle

def detect_circle_at_axis(slice_2d, center_axis, n_iter=100, threshold=0.0003, prefer_inner=True):
    """중심축 기반으로 원 검출 함수. 원의 둘레당 포인트 수를 기반으로 원을 판별 및 면적 계산."""
    if len(slice_2d) < MIN_POINTS_PER_SLICE:
        return None
    
    distances = np.linalg.norm(slice_2d - center_axis, axis=1)
    best_radius, best_score = None, -1.0
    
    for _ in range(n_iter):
        r_candidate = distances[np.random.randint(len(slice_2d))]
        if not (MIN_RADIUS <= r_candidate <= MAX_RADIUS):
            continue
        
        inlier_count = np.sum(np.abs(distances - r_candidate) < threshold)
        circumference = 2 * np.pi * r_candidate
        ppm = inlier_count / circumference if circumference > 0 else 0
        
        # 내벽 선호: 작은 반지름에 가중치 부여
        if prefer_inner:
            # 반지름이 작을수록 높은 가중치 (MAX_RADIUS 기준으로 정규화)
            radius_weight = 1.0 + (MAX_RADIUS - r_candidate) / MAX_RADIUS
            score = ppm * radius_weight
        else:
            score = ppm
        
        if score > best_score:
            best_score, best_radius = score, r_candidate
    
    # 최종 검증: 원래 ppm 기준으로도 최소 임계값 확인
    if best_radius:
        final_inlier_count = np.sum(np.abs(distances - best_radius) < threshold)
        final_circumference = 2 * np.pi * best_radius
        final_ppm = final_inlier_count / final_circumference if final_circumference > 0 else 0
        
        if final_ppm >= MIN_POINTS_PER_METER:
            return best_radius
    return None

def check_has_bottom(slice_2d, center_axis, radius, threshold=0.0003):
    """바닥이 있는지 확인하는 함수"""
    if len(slice_2d) < MIN_POINTS_PER_SLICE:
        return False
    
    distances = np.linalg.norm(slice_2d - center_axis, axis=1)
    
    # 내부 반지름 (원 반지름의 60% 이내)
    inner_radius = radius * INNER_RADIUS_RATIO
    inner_mask = distances < inner_radius
    inner_count = np.sum(inner_mask)
    
    if inner_count == 0:
        return False  # 내부 포인트가 없으면 바닥 없음
    
    # 밀도 확인만 사용
    inner_area = np.pi * inner_radius ** 2
    inner_density = inner_count / inner_area if inner_area > 0 else 0
    
    # 밀도 조건만으로 판정
    has_bottom = inner_density >= MIN_INNER_DENSITY
    
    return has_bottom

# 유틸리티 함수들
def get_slice(points, z, thickness=THICKNESS):
    """높이 z에서 슬라이스 추출"""
    mask = (points[:, 2] >= z - thickness) & (points[:, 2] <= z + thickness)
    return points[mask]

def add_volume_data(volume_data, z, radius):
    """volume_data에 단면 정보 추가"""
    volume_data.append({'z': z, 'radius': radius, 'area': np.pi * radius**2})

def search_heights(points, z_range, center_axis, volume_data, max_failures=MAX_CONSECUTIVE_FAILURES):
    """높이 범위를 탐색하며 원 검출"""
    failures = 0
    for z in z_range:
        slice_points = get_slice(points, z)
        if len(slice_points) < MIN_POINTS_PER_SLICE:
            failures += 1
            if failures >= max_failures:
                break
            continue
        
        radius = detect_circle_at_axis(slice_points[:, :2], center_axis)
        if radius is not None:
            failures = 0
            add_volume_data(volume_data, z, radius)
        else:
            failures += 1
            if failures >= max_failures:
                break


In [16]:
#json파일읽는 함수수
import json

def load_jsonl(jsonl_path):
    """JSONL 파일 읽기 - 여러 줄에 걸친 JSON 객체 처리"""
    data = []
    with open(jsonl_path, 'r') as f:
        content = f.read()
    
    # 중괄호 카운팅으로 각 JSON 객체 분리
    current_obj = ""
    brace_count = 0
    for char in content:
        current_obj += char
        if char == '{':
            brace_count += 1
        elif char == '}':
            brace_count -= 1
            if brace_count == 0:  # JSON 객체 완성
                try:
                    data.append(json.loads(current_obj.strip()))
                except json.JSONDecodeError as e:
                    print(f"JSON 파싱 오류: {e}")
                current_obj = ""
    return data


In [17]:
#ARcore 좌표계로 변환하는 함수

def transform_point_to_arcore(point_scene, first_frame_params, scene_metadata=None):
    import numpy as np
    from scipy.spatial.transform import Rotation
    
    point_scene = np.array(point_scene, dtype=np.float32)
    
    # 1. 중력 정렬 역변환 (중력 정렬 씬 → DA3 최종 씬)
    T_inv = np.eye(4)
    if scene_metadata is not None:
        A = np.array(scene_metadata.get('hf_alignment', np.eye(4))).reshape(4, 4)
        gl_to_cv = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
        arcore_y = np.array([0, 1, 0])
        gravity = (A[:3, :3] @ gl_to_cv @ arcore_y)
        gravity = gravity / np.linalg.norm(gravity)
        
        R = Rotation.align_vectors([[0, 0, 1]], [gravity])[0]
        T = np.eye(4)
        T[:3, :3] = R.as_matrix()
        T_inv = np.linalg.inv(T)
    
    point_da3_final = T_inv[:3, :3] @ point_scene + T_inv[:3, 3]
    
    # 2. DA3 최종 씬 → 원본 씬 (hf_alignment 역변환)
    if scene_metadata is not None:
        A = np.array(scene_metadata.get('hf_alignment', np.eye(4))).reshape(4, 4)
        A_inv = np.linalg.inv(A)
        # A_inv는 glTF -> CV 변환 + 원점 복원
        point_original = A_inv[:3, :3] @ point_da3_final + A_inv[:3, 3]
    else:
        point_original = point_da3_final
    
    # 3. 원본 씬 → ARCore 카메라 (첫 번째 프레임 pose)
    quat = np.array(first_frame_params['quat'])
    pos_arcore = np.array(first_frame_params['pos'])
    
    gl_to_cv = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]], dtype=np.float32)
    R_c2w_gl = Rotation.from_quat(quat).as_matrix()
    R_c2w_cv = gl_to_cv @ R_c2w_gl @ gl_to_cv.T
    pos_cv = gl_to_cv @ pos_arcore
    
    c2w = np.eye(4)
    c2w[:3, :3] = R_c2w_cv
    c2w[:3, 3] = pos_cv
    w2c = np.linalg.inv(c2w)
    
    point_camera_cv = w2c[:3, :3] @ point_original + w2c[:3, 3]
    
    # 4. OpenCV → OpenGL (ARCore 좌표계)
    cv_to_gl = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]], dtype=np.float32)
    point_arcore = cv_to_gl @ point_camera_cv
    
    return point_arcore

In [19]:
points = load_and_align_scene('/data/ephemeral/home/project/output/scene.glb')

# 전처리: 원점 근처 필터링 및 정렬
mask = np.linalg.norm(points[:, :2], axis=1) < 0.2 #원점기준 xy거리 0.2m 이하 필터링
filtered = points[mask]
sorted_idx = np.argsort(filtered[:, 2])
sorted_points, sorted_heights = filtered[sorted_idx], filtered[sorted_idx, 2]

# 1단계: 중심축 계산
z_range = np.linspace(sorted_heights.min(), sorted_heights.max(), 20) #z축 범위 20개 구간으로 나누기.
circle_data = []
origin = np.array([0, 0])

for z in z_range:
    start = np.searchsorted(sorted_heights, z - THICKNESS)
    end = np.searchsorted(sorted_heights, z + THICKNESS)
    if end - start < MIN_POINTS_PER_SLICE:
        continue
    
    slice_2d = sorted_points[start:end, :2]
    # 원점 거리 페널티 없이 순수하게 원 검출
    circle = fit_circle_ransac(slice_2d, n_iter=30, threshold=0.005, min_inliers=10)
    
    if circle:
        # 원점 거리 계산 및 final_score 계산
        dist_from_origin = np.linalg.norm(circle['center'] - origin)
        final_score = circle['score'] - dist_from_origin * 1.5
        
        circle_data.append({
            'z': z,
            'center': circle['center'],
            'radius': circle['radius'],
            'score': circle['score'],
            'final_score': final_score,
            'dist_from_origin': dist_from_origin
        })

if not circle_data:
    print("컵 원을 찾지 못했습니다.")
else:
    # final_score 기준으로 정렬하여 상위 5개 선택
    circle_data.sort(key=lambda x: x['final_score'], reverse=True)
    top_5_circles = circle_data[:5]
    center_axis = np.median([c['center'] for c in top_5_circles], axis=0) ##중심축
    reference_z = top_5_circles[0]['z'] ##부피 계산 할때 처음 탐색을 시작할 z값. final score가장 높은 단면
    
    print(f"중심축: ({center_axis[0]:.6f}, {center_axis[1]:.6f}), 기준 높이: {reference_z:.4f}m")
    
    # 2단계: 부피 계산을 위한 원 검출
    volume_data = []
    z_min, z_max = points[:, 2].min(), points[:, 2].max()
    STEP = 0.002  # 2mm 간격
    
    # 기준 높이에서 원 검출 시도
    ref_slice = get_slice(points, reference_z)
    if len(ref_slice) >= MIN_POINTS_PER_SLICE:
        ref_radius = detect_circle_at_axis(ref_slice[:, :2], center_axis)
        if ref_radius:
            add_volume_data(volume_data, reference_z, ref_radius)
    
    # 위쪽 탐색: 기준 높이 + 0.002, + 0.004, ... (z_max까지)
    z_above = np.arange(reference_z + STEP, z_max + STEP, STEP)
    search_heights(points, z_above, center_axis, volume_data)
    
    # 아래쪽 탐색: 기준 높이 - 0.002, - 0.004, ... (z_min까지)
    z_below = np.arange(reference_z - STEP, z_min - STEP, -STEP)
    search_heights(points, z_below, center_axis, volume_data)
    


    # 3단계: 하위 20% 범위에서만 바닥 검출
    if volume_data:
        volume_data.sort(key=lambda x: x['z'])
        
        # 높이 범위 계산
        height_range = volume_data[-1]['z'] - volume_data[0]['z']
        bottom_20_percent_range = height_range * BOTTOM_SEARCH_RATIO
        bottom_20_end_z = volume_data[0]['z'] + bottom_20_percent_range
        
        print(f"\n바닥 검출 범위: {volume_data[0]['z']:.4f}m ~ {bottom_20_end_z:.4f}m (하위 20%)")
        
        # 하위 20% 범위의 데이터만 바닥 검출
        bottom_detected_heights = []
        for data in volume_data:
            z = data['z']
            # 하위 20% 범위에 있는지 확인
            if z <= bottom_20_end_z:
                radius = data['radius']
                slice_points = get_slice(points, z)
                
                if len(slice_points) >= MIN_POINTS_PER_SLICE:
                    has_bottom = check_has_bottom(slice_points[:, :2], center_axis, radius)
                    if has_bottom:
                        bottom_detected_heights.append(z)
        
        # 바닥이 있는 가장 높은 높이 찾기
        if bottom_detected_heights:
            bottom_end_z = max(bottom_detected_heights)
            print(f"바닥 검출: {len(bottom_detected_heights)}개 높이에서 바닥 발견")
            print(f"바닥 끝 높이: {bottom_end_z:.4f}m")
            
            # 바닥이 아닌 부분만 필터링 (바닥 끝 높이보다 위쪽만)
            volume_data_filtered = [data for data in volume_data if data['z'] > bottom_end_z]
            
            print(f"필터링 전: {len(volume_data)}개 단면")
            print(f"필터링 후: {len(volume_data_filtered)}개 단면 (바닥 제외)")
            
            if len(volume_data_filtered) > 1:
                volume_data = volume_data_filtered
            else:
                print("경고: 바닥을 제외하면 단면이 부족합니다. 전체 데이터 사용.")
        else:
            print("하위 20% 범위에서 바닥이 검출되지 않았습니다. 전체 데이터로 부피 계산합니다.")


    # 4단계: 부피 계산
    if volume_data:
        volume_data.sort(key=lambda x: x['z'])
        total_volume = sum((volume_data[i]['area'] + volume_data[i+1]['area']) / 2 * 
                          (volume_data[i+1]['z'] - volume_data[i]['z']) 
                          for i in range(len(volume_data) - 1))
        
        print(f"\n단면 개수: {len(volume_data)}개")
        print(f"높이: {volume_data[0]['z']:.4f}m ~ {volume_data[-1]['z']:.4f}m")
        print(f"부피: {total_volume * 1000:.2f} mL ({total_volume * 1e6:.2f} cm³)")
    else:
        print("원을 찾지 못했습니다.")

중심축: (0.001418, 0.034405), 기준 높이: 0.0965m

바닥 검출 범위: -0.0295m ~ -0.0031m (하위 20%)
하위 20% 범위에서 바닥이 검출되지 않았습니다. 전체 데이터로 부피 계산합니다.

단면 개수: 65개
높이: -0.0295m ~ 0.1025m
부피: 0.82 mL (817.42 cm³)


In [21]:
#메인로직
import trimesh
import numpy as np
import glob
import os

# 1. GLB 파일 로드 (씬 메타데이터 포함)
scene = trimesh.load('/data/ephemeral/home/project/output/scene.glb')
scene_metadata = scene.metadata  # hf_alignment 포함

# 2. JSONL 파일 로드 (ARCore pose 데이터)
# ar_folder 내 가장 최근 session 폴더 찾기
base_folder = "/data/ephemeral/home/ar_folder"
session_folders = sorted(glob.glob(os.path.join(base_folder, "session_*")))

if not session_folders:
    raise FileNotFoundError(f"세션 폴더를 찾을 수 없습니다: {os.path.join(base_folder, 'session_*')}")

session_folder = session_folders[-1]  # 가장 최근 세션 사용
print(f"사용할 세션 폴더: {session_folder}")

# JSONL 파일 찾기 (세션 폴더 안에서)
jsonl_files = glob.glob(os.path.join(session_folder, "*.jsonl"))
if not jsonl_files:
    raise FileNotFoundError(f"JSONL 파일을 찾을 수 없습니다: {os.path.join(session_folder, '*.jsonl')}")

if len(jsonl_files) > 1:
    print(f"경고: 여러 개의 JSONL 파일이 발견되었습니다. 첫 번째 파일을 사용합니다: {jsonl_files[0]}")

jsonl_path = jsonl_files[0]
print(f"사용할 JSONL 파일: {jsonl_path}")

jsonl_data = load_jsonl(jsonl_path)

# 3. 첫 번째 프레임 pose 가져오기 (시간순 정렬)
jsonl_data_sorted = sorted(jsonl_data, key=lambda x: x['t_ns'])
first_frame_params = jsonl_data_sorted[0]

# 4. 씬 좌표계의 점 (중력 정렬된 씬 좌표계)
point_scene = np.array([0.001418, 0.034405, 0.0])  # [x, y, z]

# 5. ARCore 좌표계로 변환
point_arcore = transform_point_to_arcore(
    point_scene=point_scene,
    first_frame_params=first_frame_params,
    scene_metadata=scene_metadata
)

print(f"씬 좌표계: {point_scene}")
print(f"ARCore 좌표계: {point_arcore}")

사용할 세션 폴더: /data/ephemeral/home/ar_folder/session_1767930970353
사용할 JSONL 파일: /data/ephemeral/home/ar_folder/session_1767930970353/session_1767930970353.jsonl
씬 좌표계: [0.001418 0.034405 0.      ]
ARCore 좌표계: [ 0.03609084 -0.01370893 -0.40844   ]
